### Using AWS SageMaker for Inference
On AWS sagemaker, using a pretrained Object Detection Model works best with tensorflow 2.8 and needs an updated version of sagemaker to use this version of tensorflow.

In [2]:
!pip install tensorflow_hub
%pip install -U sagemaker
!pip install tensorflow==2.8

  Using cached tensorflow_hub-0.12.0-py2.py3-none-any.whl (108 kB)
You should consider upgrading via the '/usr/local/bin/python3.8 -m pip install --upgrade pip' command.
     |████████████████████████████████| 534 kB 31.2 MB/s            
  Preparing metadata (setup.py) ... done
  Using cached attrs-20.3.0-py2.py3-none-any.whl (49 kB)
  Created wheel for sagemaker: filename=sagemaker-2.91.1-py2.py3-none-any.whl size=737694 sha256=71a1da6a1509d92fd87ecf1905789b5667ecf865457037af7740697977b41c5e
  Stored in directory: /root/.cache/pip/wheels/85/39/68/a7f8e2b484c8c9b492cc88d256ae7ef71de803467bc4c0801e
Successfully built sagemaker
  Attempting uninstall: attrs
    Found existing installation: attrs 21.2.0
    Uninstalling attrs-21.2.0:
      Successfully uninstalled attrs-21.2.0
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.70.0
    Uninstalling sagemaker-2.70.0:
      Successfully uninstalled sagemaker-2.70.0
You should consider upgrading via the '/usr/loc

Import the necessary libraries

In [3]:
import os
import pathlib

import matplotlib
import matplotlib.pyplot as plt

import io
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from six.moves.urllib.request import urlopen

import tensorflow as tf
import tensorflow_hub as hub
import sagemaker
import json
import cv2
tf.get_logger().setLevel('ERROR')

In [4]:
sess = sagemaker.Session()
role = sagemaker.get_execution_role()
dataset_bucket = "swifty-datasets"

bucket = sess.default_bucket()

print("Bucket: {}".format(bucket))
print("SageMaker ver: " + sagemaker.__version__)
print("Tensorflow ver: " + tf.__version__)

Bucket: sagemaker-us-east-1-038469568353
SageMaker ver: 2.91.1
Tensorflow ver: 2.8.0


In [6]:
from sagemaker.tensorflow import TensorFlowModel

model = TensorFlowModel(model_data='s3://swifty-ai-models/other_models/object_detector.tar.gz', role=role,framework_version = '2.8')

predictor = model.deploy(initial_instance_count=1, instance_type='ml.m5.xlarge', endpoint_name='object-detection-endpoint-1')

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


-----!

In [7]:
#Send the copy of the image arrat 
def load_image_into_numpy_array(path):
  """Load an image from file into a numpy array.

  Puts image into numpy array to feed into tensorflow graph.
  Note that by convention we put it into a numpy array with shape
  (height, width, channels), where channels=3 for RGB.

  Args:
    path: the file path to the image

  Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
  """
  image = None
  if(path.startswith('https')):
    response = urlopen(path)
    image_data = response.read()
    image_data = BytesIO(image_data)
    image = Image.open(image_data)
  else:
    image_data = tf.io.gfile.GFile(path, 'rb').read()
    image = Image.open(BytesIO(image_data)) 
    #resize and sent this image as array to endpoint for inference
  basewidth = 128 #Resize to basewidth of 800 and resize it proportionally to the height
  wpercent = (basewidth/float(image.size[0]))
  hsize = int((float(image.size[1])*float(wpercent)))
  image = image.resize((basewidth,hsize), Image.ANTIALIAS)
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (1, im_height, im_width, 3)).astype(np.uint8)

In [8]:
image_path = 'https://upload.wikimedia.org/wikipedia/commons/0/09/The_smaller_British_birds_%288053836633%29.jpg' 

In [9]:
input_tensor = load_image_into_numpy_array(image_path)

In [52]:
#Send this to object detection model
input_data = {
  'instances': input_tensor.tolist()
}

In [62]:
data = json.dumps({"instances":input_tensor.tolist()})

In [ ]:
print(input)

In [11]:
import requests

In [94]:
#invoke API
headers = {"content-type":"application/json"}
json_response = requests.post('https://p5dv58hpub.execute-api.us-east-1.amazonaws.com/InitialStage/objectdataprediction',data=data,headers = headers)
result = json.loads(json_response.text)

In [95]:
print(json_response)

<Response [200]>


In [98]:
print(result['predictions'][0]['detection_classes'])
print(result['predictions'][0]['detection_scores'])

[64.0, 72.0, 64.0, 86.0, 64.0, 1.0, 86.0, 84.0, 64.0, 88.0, 64.0, 64.0, 41.0, 64.0, 43.0, 65.0, 74.0, 88.0, 1.0, 1.0, 64.0, 19.0, 64.0, 1.0, 41.0, 16.0, 40.0, 16.0, 16.0, 64.0, 1.0, 1.0, 90.0, 86.0, 88.0, 16.0, 64.0, 74.0, 41.0, 62.0, 16.0, 38.0, 73.0, 33.0, 85.0, 87.0, 77.0, 82.0, 18.0, 1.0, 61.0, 64.0, 64.0, 16.0, 67.0, 41.0, 34.0, 84.0, 17.0, 52.0, 16.0, 18.0, 84.0, 85.0, 62.0, 63.0, 1.0, 84.0, 31.0, 86.0, 64.0, 28.0, 51.0, 16.0, 16.0, 64.0, 51.0, 16.0, 16.0, 84.0, 84.0, 16.0, 15.0, 50.0, 17.0, 64.0, 3.0, 87.0, 31.0, 64.0, 16.0, 86.0, 31.0, 62.0, 27.0, 17.0, 87.0, 1.0, 16.0, 51.0]
[0.517624438, 0.454195559, 0.432849, 0.414067984, 0.409557164, 0.399034381, 0.390559494, 0.389045477, 0.347824693, 0.316123426, 0.305510849, 0.304259121, 0.303778112, 0.295902461, 0.291772664, 0.285102516, 0.275288075, 0.274885565, 0.2744039, 0.268493295, 0.261832327, 0.25850141, 0.252827942, 0.249252647, 0.248189956, 0.248147368, 0.240350157, 0.238704413, 0.236564338, 0.225014, 0.224780172, 0.22082445, 0.

In [118]:
result = predictor.predict(input)

In [155]:
detection_scores= result['predictions'][0]['detection_scores']
detection_classes= result['predictions'][0]['detection_classes']

In [120]:
print(result['predictions'][0]['detection_classes'])

[16.0, 84.0, 86.0, 86.0, 64.0, 72.0, 16.0, 88.0, 16.0, 16.0, 5.0, 16.0, 65.0, 16.0, 84.0, 82.0, 16.0, 85.0, 1.0, 1.0, 16.0, 17.0, 64.0, 88.0, 16.0, 86.0, 19.0, 86.0, 38.0, 84.0, 16.0, 1.0, 16.0, 16.0, 51.0, 64.0, 16.0, 61.0, 16.0, 84.0, 16.0, 3.0, 16.0, 42.0, 16.0, 84.0, 50.0, 16.0, 51.0, 16.0, 48.0, 3.0, 16.0, 1.0, 44.0, 16.0, 24.0, 1.0, 16.0, 51.0, 16.0, 28.0, 88.0, 61.0, 84.0, 28.0, 34.0, 64.0, 48.0, 1.0, 62.0, 84.0, 73.0, 16.0, 64.0, 24.0, 1.0, 17.0, 5.0, 64.0, 16.0, 50.0, 85.0, 84.0, 16.0, 64.0, 86.0, 64.0, 67.0, 87.0, 43.0, 64.0, 1.0, 28.0, 16.0, 44.0, 16.0, 8.0, 84.0, 1.0]


In [105]:
def unique(list1):
     
    # insert the list to the set
    list_set = set(list1)
    # convert the set to the list
    unique_list = (list(list_set))
    print(unique_list)

In [106]:
unique(result['predictions'][0]['detection_classes'])

[1.0, 3.0, 5.0, 8.0, 16.0, 17.0, 19.0, 24.0, 28.0, 34.0, 38.0, 42.0, 43.0, 44.0, 48.0, 50.0, 51.0, 61.0, 62.0, 64.0, 65.0, 67.0, 72.0, 73.0, 82.0, 84.0, 85.0, 86.0, 87.0, 88.0]


In [145]:
print(len((result['predictions'][0]['detection_scores'])))
print(len((result['predictions'][0]['detection_classes'])))

100
100


In [146]:
txt_file = open("cococlasses.txt", "r")
content = txt_file.read().split("\n")
print(content)

['background', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'street sign', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'hat', 'backpack', 'umbrella', 'shoe', 'eye glasses', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'plate', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'mirror', 'dining table', 'window', 'desk', 'toilet', 'door', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'blender', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush', 'hair brush']


In [139]:
print(len(content))

92


In [141]:
print(content[16])

bird


In [150]:
def convertToInt(list1):
    detected_classes=[]
    for i in range(len(list1)):
        detected_classes.append(int(list1[i]))
    return detected_classes 

In [151]:
detected_classes = convertToInt(result['predictions'][0]['detection_classes'])

In [159]:
def findTop10Classes(list1, list2, content):
    print("The system has detected the following objects:")
    print("Object \t\t\t Confidence")
    for i in range(10):
        print(content[list1[i]].capitalize(),"\t\t\t",list2[i]*100)

In [160]:
findTop10Classes(detected_classes,detection_scores,content)

The system has detected the following objects:
Object 			 Confidence
Bird 			 64.2007589
Book 			 48.472
Vase 			 46.1754799
Vase 			 45.5887854
Potted plant 			 44.4863498
Tv 			 43.046379099999996
Bird 			 40.3677404
Teddy bear 			 39.5147175
Bird 			 37.4859095
Bird 			 35.2080941


### Delete End Point

In [12]:
sess.delete_endpoint(predictor.endpoint)

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


### Without using AWS services

In [ ]:
detector = hub.load("https://tfhub.dev/tensorflow/ssd_mobilenet_v2/2")

In [ ]:
image_path = 'https://upload.wikimedia.org/wikipedia/commons/0/09/The_smaller_British_birds_%288053836633%29.jpg'
image_np = load_image_into_numpy_array(image_path)

plt.figure(figsize=(24,32))
plt.imshow(image_np[0])
plt.show()


In [ ]:
# running inference
results = detector(image_np)
 #needs tf 2.8 to work

result = {key:value.numpy() for key,value in results.items()}
print(result.keys())

In [ ]:
print(image_np)

In [ ]:
print(result['detection_scores'])

In [ ]:
print(result['detection_classes'])
#Get image,
#Predict and return output from end point
#After predicting, Find way to add detection boxes to output and then save to AWS s3 on the image.